# Redes Neurais | Trabalho Final da disciplina de Aprendizado de Máquina INE5664

# Implementação de uma Rede Neural Multitarefa

## Introdução
Este notebook apresenta a implementação de uma rede neural customizada capaz de lidar com tarefas de:
1. **Classificação Binária**
2. **Regressão**
3. **Classificação Multiclasse**

Ele inclui funções de ativação, funções de perda, retropropagação e um modelo de rede neural treinável, com dados simulados ou reais.

---

## Importação de Bibliotecas Necessárias
As bibliotecas são fundamentais para manipulação de dados, cálculos e avaliação do modelo.


In [ ]:
import warnings
warnings.filterwarnings('ignore') ## Desativa os avisos exibidos durante a execução

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score ## Define as Métricas de Avaliação
from sklearn.model_selection import train_test_split ## Divide o conjunto de dados em conjuntos de treino e teste
from sklearn.preprocessing import OneHotEncoder, StandardScaler ## Normaliza os dados para garantir que tenham média 0 e desvio padrão 1


1. **Warnings**: usada para controlar alertas gerados pelo Python durante a execução do código.
2. **Pandas**: usada para manipulação e análise de dados. Amplamente utilizada em projetos de aprendizado de máquina devido à sua capacidade de trabalhar com grandes conjuntos de dados de forma eficiente.
3. **NumPy**:  base fundamental para cálculos numéricos e operações matemáticas em Python.
4. **Scikit-learn**:  fornece ferramentas simples e eficientes para aprendizado de máquina.

---

## Métricas de avaliação do projeto

- **mean_squared_error**: Mede o erro médio ao quadrado entre os valores reais e previstos (usado em regressão).
- **r2_score**: Mede o quão bem os dados se ajustam ao modelo (valores próximos de 1 indicam bom ajuste).
- **accuracy_score**: Mede a porcentagem de previsões corretas (usado em classificação).

---

## Definição das Funções de Ativação

Funções de ativação determinam a saída de um neurônio com base nos dados de entrada e nos pesos aplicados. Em outras palavras, as funções de ativação introduzem não-linearidade no modelo, permitindo que ele aprenda e represente relações complexas entre os dados de entrada e saída.

### ReLU

In [ ]:
# Funções de Ativação
def relu(z):
    return np.maximum(0, z)

- **Descrição**: A função ReLU retorna o valor de entrada diretamente se for positivo; caso contrário, retorna zero.

- **Propósito**: Introduz não-linearidade mantendo simplicidade computacional. É amplamente usada em camadas ocultas de redes neurais profundas.

---

### Sigmoid

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

- **Descrição**: A função Sigmoid transforma qualquer valor real em um intervalo entre 0 e 1, funcionando como uma probabilidade.

- **Propósito**: Ideal para problemas de classificação binária, onde a saída deve ser interpretada como uma probabilidade.

---

### Softmax

In [ ]:
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

- **Descrição**: A função Softmax converte um vetor de valores em uma distribuição de probabilidades. Cada saída é normalizada para somar 1.

- **Propósito**: Usada em problemas de classificação multiclasse para calcular a probabilidade de cada classe.

---

### Linear

In [ ]:
def linear(z):
    return z

- **Descrição**: A função Linear retorna a entrada sem modificações, ou seja, f(z)=z.

- **Propósito**: Usada em problemas de classificação multiclasse para calcular a probabilidade de cada classe.

---

## Derivadas das Funções de Ativação

As derivadas são essenciais para o cálculo de gradientes na retropropagação.

In [ ]:
# Derivadas das Funções de Ativação
def relu_derivative(z):
    return (z > 0).astype(float)

# Dicionários de funções para fácil acesso
activation_functions = {
    'relu': relu,
    'sigmoid': sigmoid,
    'softmax': softmax,
    'linear': linear
}

activation_derivatives = {
    'relu': relu_derivative
}

---

## Funções de Perda

As funções de perda medem a discrepância entre a saída do modelo e os valores reais.

- **Binary Loss**: Usada para classificação binária.
- **Regression Loss**: Usada para regressão.
- **Multiclass Loss**: Usada para classificação multiclasse.

In [ ]:
# Funções de Perda
def binary_loss(y, output):
    epsilon = 1e-15
    output = np.clip(output, epsilon, 1 - epsilon)
    return -np.mean(y * np.log(output) + (1 - y) * np.log(1 - output))

def regression_loss(y, output):
    return np.mean((output - y) ** 2)

def multiclass_loss(y, output):
    epsilon = 1e-15
    output = np.clip(output, epsilon, 1 - epsilon)
    y = y.astype(int)
    log_probs = -np.log(output[np.arange(y.shape[0]), y.flatten()])
    return np.mean(log_probs)

# Dicionário de funções de perda
loss_functions = {
    'binary': binary_loss,
    'regression': regression_loss,
    'multiclass': multiclass_loss
}

### y

- **Ground Truth (Valor Real)**
- É o conjunto de valores reais (ou verdadeiros) associados a cada exemplo do conjunto de dados. Representa o que esperamos que o modelo preveja.
- Estes valores vêm do dataset usado para treinamento, validação ou teste. Geralmente são os rótulos de classe para problemas de classificação ou valores contínuos para regressão.

### output

- **Previsões do Modelo**
- É o conjunto de valores previstos pelo modelo em resposta a uma entrada específica.
- Esses valores são gerados pelo passo forward da rede neural, aplicando funções de ativação na saída dos neurônios.

---

## Implementação da Classe NeuralNetwork

A classe `NeuralNetwork` encapsula:
- Inicialização dos pesos.
- Forward propagation.
- Backward propagation (retropropagação).
- Treinamento.
- Avaliação.

### def __init__

Esse método inicializa os atributos da rede neural e define os **parâmetros** necessários para configurá-la:

- **input_size**: Número de neurônios na camada de entrada, ou seja, o número de características do conjunto de dados (dimensão dos dados de entrada).

- **hidden_neurons**: Número de neurônios na camada oculta. Este número afeta a capacidade da rede de capturar padrões nos dados.

- **output_neurons**: Número de neurônios na camada de saída. Geralmente corresponde ao número de classes (classificação) ou à dimensão da saída (regressão).

- **activation**: Nome da função de ativação ('relu', 'sigmoid', etc.). Esta função é usada para calcular a saída dos neurônios na camada oculta.

- **loss**: Nome da função de perda ('binary', 'regression', 'multiclass'). Define como a rede avaliará a diferença entre as previsões e os valores reais.

- **learning_rate**: Taxa de aprendizado usada para atualizar os pesos durante a otimização.

- **task_type**: Tipo de tarefa que a rede executará ('binary', 'regression' ou 'multiclass'). Determina o comportamento da função forward e da função de perda.

In [ ]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_neurons, output_neurons, activation, loss, learning_rate, task_type):
        self.input_size = input_size
        self.hidden_neurons = hidden_neurons
        self.output_neurons = output_neurons
        self.activation = activation_functions[activation]
        self.activation_derivative = activation_derivatives.get(activation)
        self.loss = loss_functions[loss]
        self.learning_rate = learning_rate
        self.task_type = task_type

#### Inicialização dos pesos

O trecho de inicialização dos pesos no método __init__ da classe NeuralNetwork é essencial para garantir que a rede neural comece o treinamento com valores adequados para uma boa convergência.

1. **Pesos (W)**:

- São os parâmetros ajustáveis da rede que conectam os neurônios entre as camadas.
- Determinam a força e a direção da influência de uma entrada ou de uma camada sobre a próxima.

2. **Bias (B)**:

- São valores adicionados ao somatório ponderado das entradas antes de passar pela função de ativação.
- Permitem à rede neural se ajustar melhor ao deslocamento dos dados e evitar ser limitada a passar pela origem.

In [ ]:
# Inicialização dos pesos
self.W1 = np.random.randn(input_size, hidden_neurons) * np.sqrt(2 / input_size)
self.B1 = np.zeros((1, hidden_neurons))
self.W2 = np.random.randn(hidden_neurons, output_neurons) * np.sqrt(2 / hidden_neurons)
self.B2 = np.zeros((1, output_neurons))


---

### Método forward

O método forward da classe NeuralNetwork implementa a propagação para frente (forward propagation), que é o processo de calcular a saída da rede neural para uma determinada entrada. Ele usa as entradas, os pesos e os bias para calcular as ativações de cada camada até chegar à saída final. 

In [ ]:
 def forward(self, X):
        # X: Dados de entrada (matriz de dimensão [n_amostras, n_características])
        # W1: Pesos conectando a entrada à camada oculta
        # B1: Bias da camada oculta (vetor de dimensão [1, n_neurônios_ocultos])
        self.z1 = X.dot(self.W1) + self.B1

        # Aplica a função de ativação escolhida na camada oculta
        # activation: Função como ReLU, sigmoid, etc.
        self.f1 = self.activation(self.z1)

         # Calcula a entrada para a camada de saída (pré-ativação)
        # f1: Saída da camada oculta (dimensão [n_amostras, n_neurônios_ocultos])
        # W2: Pesos conectando a camada oculta à camada de saída
        # B2: Bias da camada de saída (dimensão [1, n_neurônios_saida])
        self.z2 = self.f1.dot(self.W2) + self.B2

        # Decide como processar a saída da rede com base no tipo de tarefa
        if self.task_type == 'binary':
            return sigmoid(self.z2)
        elif self.task_type == 'regression':
            return self.z2
        elif self.task_type == 'multiclass':
            return softmax(self.z2)


---

### Método Backward

Ele realiza a propagação para trás (backpropagation), o processo usado para calcular os gradientes da função de perda em relação aos pesos da rede, permitindo ajustar os pesos durante o treinamento.

#### Erros na camada de saída (delta2)

Depende do tipo de tarefa:

- **Binária**: diferença entre a saída prevista (output) e o rótulo real (y)
- **Regressão**: diferença contínua entre a saída e os rótulos
- **Multiclasse**: subtrai 1 da probabilidade da classe verdadeira

In [ ]:
def backward(self, X, y, output):
    # Número de exemplos no lote de entrada
    m = X.shape[0] 

    # Calcula o erro na camada de saída (delta2) com base no tipo de tarefa
    if self.task_type == 'binary':
        delta2 = output - y
    elif self.task_type == 'regression':
        y = y.reshape(-1, self.output_neurons)  # Garante que `y` tenha a forma esperada
        delta2 = output - y
    elif self.task_type == 'multiclass':
        delta2 = output  # Inicialmente, `delta2` é a saída
        delta2[np.arange(m), y.flatten()] -= 1  # Ajusta para refletir o erro da classe correta


No método backward, após calcular a saída da rede e o erro em relação aos valores reais, **os gradientes são calculados para ajustar os parâmetros da rede neural (pesos e biases)**.

Esses gradientes indicam a direção e a magnitude da modificação necessária para minimizar a função de perda.

1. **dW2**

- Gradiente da perda em relação aos pesos da camada de saída.
- Representa como os pesos devem ser ajustados para reduzir o erro.

2. **dB2**

- Gradiente da perda em relação aos biases da camada de saída.
- Representa como os biases devem ser ajustados para reduzir o erro.

In [ ]:
    dW2 = self.f1.T.dot(delta2) / m  # Gradiente do peso: derivada de perda em relação a W2
    dB2 = np.sum(delta2, axis=0, keepdims=True) / m  # Gradiente do bias: média do erro na saída

Agora, o trecho seguinte do código faz parte do cálculo do gradiente da camada oculta durante a etapa de retropropagação (backpropagation). Ele **calcula como o erro na camada de saída influencia a camada oculta**, permitindo que os pesos e biases dessa camada também sejam ajustados para reduzir a perda.

In [ ]:
    # Calcula o erro propagado para a camada oculta (delta1)
    if self.activation_derivative:
        # Propaga o erro da camada de saída para a camada oculta
        # Ajusta o erro considerando a função de ativação
        delta1 = delta2.dot(self.W2.T) * self.activation_derivative(self.z1)
    else:
        # Se a função de ativação não tiver derivada definida, apenas propaga o erro
        delta1 = delta2.dot(self.W2.T)

Sendo assim, agora são definidos os gradiente da camada oculta.

1. **dW1**: Gradiente dos pesos da camada oculta; calcula como cada peso contribui para o erro, considerando a entrada X.

2. **dB1**: Gradiente dos biases da camada oculta; reflete o erro acumulado de cada neurônio.

Esses gradientes são a base para ajustar os parâmetros do modelo, reduzindo a função de perda e melhorando a performance.

In [ ]:
  # Gradiente dos pesos e bias da camada oculta (W1 e B1)
    dW1 = X.T.dot(delta1) / m  # Gradiente do peso: derivada de perda em relação a W1
    dB1 = np.sum(delta1, axis=0, keepdims=True) / m  # Gradiente do bias: média do erro na camada oculta

Na última parte desse método, é realizada a **atualização dos pesos e biases do modelo após o cálculo dos gradientes**, com o objetivo de minimizar o erro (perda) durante o treinamento.

- Cada parâmetro (peso W e bias B) é ajustado subtraindo o gradiente multiplicado pela taxa de aprendizado (learning_rate).
- Isso move os parâmetros na direção que diminui o erro.
- A atualização busca reduzir a perda, ajustando os parâmetros para melhorar a predição do modelo, baseado nos gradientes calculados durante a retropropagação.

In [ ]:
    # Atualização dos pesos
    self.W1 -= self.learning_rate * dW1
    self.B1 -= self.learning_rate * dB1
    self.W2 -= self.learning_rate * dW2
    self.B2 -= self.learning_rate * dB2

---

### Método train

A função train é responsável por treinar o modelo de rede neural, iterando sobre os dados de treinamento por um número específico de épocas. Ela realiza a propagação para frente (forward), calcula a perda (loss) e faz a retropropagação (backward) para atualizar os pesos e biases.

1. **Loop de épocas**

- O treinamento ocorre por várias épocas (iterações), definidas pelo parâmetro epochs.
- Em cada época, o modelo passa pelos seguintes passos:

2. **Propagação para frente (Forward)**

- A função `self.forward(X_train)` é chamada para calcular a saída do modelo com os dados de treinamento.
- A saída é comparada com a saída real (y_train), e a função de perda calcula o erro.

3. **Cálculo da perda (Loss)**

- A perda é calculada com a função `self.loss(y_train, output)`, que mede a diferença entre a previsão do modelo e o valor real.
- A função de perda depende do tipo de tarefa (classificação binária, regressão ou classificação multiclasse).

4. **Retropropagação (Backward)**

- A função `self.backward(X_train, y_train, output)` é chamada para calcular os gradientes da perda em relação aos pesos e biases.
- Esses gradientes são usados para ajustar os parâmetros do modelo (pesos e biases).

5. **Validação**

- O modelo é avaliado usando o conjunto de dados de validação (X_val, y_val) para verificar se está generalizando bem.
- A perda no conjunto de validação é calculada e armazenada em val_loss.

6. **Exibição do progresso**

- Se verbose for True e a época for múltiplo de 300, o código exibe a perda de treinamento e de validação para monitorar o progresso do treinamento.

In [ ]:
def train(self, X_train, y_train, X_val, y_val, epochs, verbose=True):
    for epoch in range(epochs):
        # Passo 1: Propagação para frente (Forward)
        output = self.forward(X_train)
        loss = self.loss(y_train, output)  # Calcula a perda (erro)

        # Passo 2: Retropropagação (Backward)
        self.backward(X_train, y_train, output)  # Ajusta os pesos com base no erro

        # Passo 3: Avaliação do modelo com dados de validação
        val_output = self.forward(X_val)
        val_loss = self.loss(y_val, val_output)  # Perda no conjunto de validação

        # Exibe informações a cada 300 épocas (caso verbose seja True)
        if verbose and epoch % 300 == 0:
            print(f"Época {epoch+1}/{epochs}, Perda Treino: {loss:.4f}, Perda Validação: {val_loss:.4f}")


---

### Método evaluate

A função evaluate é responsável por avaliar o desempenho do modelo após o treinamento, utilizando os dados de teste.

1. **Calcula as previsões:** A função chama self.forward(X_test) para obter as previsões do modelo usando os dados de teste (X_test).

2. **Aí, dependendo da tarefa**:

- **Classificação Binária:** Se for uma tarefa de classificação binária, a previsão é convertida para 0 ou 1 (usando um limiar de 0.5). A acurácia é então calculada e exibida.

- **Regressão:** Para regressão, as previsões são comparadas com os valores reais (de teste). São calculados o erro médio quadrático (MSE) e o R², que medem a qualidade da previsão.

- **Classificação Multiclasse:** Para a classificação multiclasse, a previsão é transformada em um índice de classe (com argmax), e a acurácia é calculada.

3. **Exibe o resultado**: Dependendo da tarefa, a função imprime a acurácia ou as métricas de regressão (MSE e R²).

In [ ]:
def evaluate(self, X_test, y_test, y_mean=None, y_std=None):
    # Passo 1: Obter previsões do modelo para os dados de teste
    predictions = self.forward(X_test)

    # Se a tarefa for de classificação binária:
    if self.task_type == 'binary':
        # Converter as previsões para 0 ou 1 com base no limiar de 0.5
        predictions = (predictions > 0.5).astype(int)
        # Calcular a acurácia comparando as previsões com os valores reais
        accuracy = accuracy_score(y_test, predictions)
        # Imprimir a acurácia e retornar o valor
        print(f"Acurácia: {accuracy:.4f}")
        return accuracy

    # Se a tarefa for de regressão:
    if self.task_type == 'regression':
        # Se os valores de y foram normalizados, denormalizar as previsões e os valores reais
        predictions = predictions * y_std + y_mean
        y_test = y_test * y_std + y_mean
        # Calcular o MSE e o R² para avaliar o desempenho do modelo
        mse = mean_squared_error(y_test, predictions)
        r2 = r2_score(y_test, predictions)
        # Imprimir o MSE e o R² e retornar os valores
        print(f"MSE: {mse:.4f}, R² Score: {r2:.4f}")
        return mse, r2

    # Se a tarefa for de classificação multiclasse:
    elif self.task_type == 'multiclass':
        # Para cada previsão, pegar o índice da classe com a maior probabilidade
        predictions = np.argmax(predictions, axis=1)
        # Calcular a acurácia para comparar as previsões com os valores reais
        accuracy = accuracy_score(y_test, predictions)
        # Imprimir a acurácia e retornar o valor
        print(f"Acurácia: {accuracy:.4f}")
        return accuracy


---

### Método load_data

A função load_data é responsável por **carregar e pré-processar os dados a partir de um arquivo CSV**.

Ela prepara os dados para serem usados no treinamento e avaliação de um modelo de aprendizado de máquina. A função adapta o processo de carregamento conforme o tipo da tarefa (task_type), que pode ser classificação binária, regressão, ou classificação multiclasse.

1. **Carregar os dados**: A função começa carregando os dados de um arquivo CSV utilizando pd.read_csv(file_path).

2. **Tarefa binária (task_type == 'binary')**:

- As variáveis independentes (entradas) e dependentes (saídas) são separadas.
- A variável de saída (y) é transformada para uma matriz coluna.
- Os dados são divididos em treino e teste (80% treino, 20% teste).
- As variáveis de entrada são normalizadas com StandardScaler.
- Retorna os dados de treino e teste.

3. **Tarefa de regressão (task_type == 'regression')**:

- Seleciona as variáveis de entrada e saída (por exemplo, TV, Rádio, Jornal e Vendas).
- As variáveis de entrada e saída são normalizadas separadamente.
- Divide os dados em treino e teste (80% treino, 20% teste).
- Retorna os dados de treino e teste com as médias e desvios padrão para reverter a normalização.

4. **Tarefa multiclasse (task_type == 'multiclass')**:

- A coluna de classes é codificada numericamente.
- As variáveis de entrada são normalizadas.
- Divide os dados em treino e teste (80% treino, 20% teste), mantendo a proporção das classes.
- Retorna os dados de treino e teste com a codificação das classes.

In [ ]:
# Função para carregar os dados
def load_data(file_path, task_type):
    data = pd.read_csv(file_path)  # Carrega os dados do CSV
    if task_type == 'binary':  # Se a tarefa for binária
        X = data.iloc[:, :-1].values  # Extrai as variáveis independentes
        y = data.iloc[:, -1].values  # Extrai a variável dependente

        y = y.reshape(-1, 1)  # Ajusta a forma de y

        # Divide os dados em treino e teste (80/20)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Normaliza os dados de entrada
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        return X_train, X_test, y_train, y_test, None, None  # Retorna os dados normalizados

    elif task_type == 'regression':  # Se a tarefa for de regressão
        dados_multi = data[['TV', 'Radio', 'Newspaper', 'Sales']]  # Seleciona as variáveis relevantes
        X_multi = dados_multi[['TV', 'Radio', 'Newspaper']].values  # Variáveis independentes
        y_multi = dados_multi['Sales'].values  # Variável dependente
        
        # Normaliza as variáveis de entrada (X) e saída (y)
        scaler_X = StandardScaler()
        X_multi_scaled = scaler_X.fit_transform(X_multi)
        scaler_y = StandardScaler()
        y_multi_scaled = scaler_y.fit_transform(y_multi.reshape(-1, 1)).flatten()

        # Divide os dados em treino e teste
        X_train, X_test, y_train, y_test = train_test_split(
            X_multi_scaled, y_multi_scaled, test_size=0.2, random_state=0
        )
        return X_train, X_test, y_train, y_test, scaler_y.mean_[0], scaler_y.scale_[0]  # Retorna os dados e parâmetros de normalização

    elif task_type == 'multiclass':  # Se a tarefa for multiclasse
        target = data.pop('Species')  # Extrai a coluna de classes (ajustar conforme necessário)
        class_map = {label: idx for idx, label in enumerate(target.unique())}  # Mapeia as classes para números
        target_encoded = target.map(class_map).values  # Codifica as classes

        # Normaliza as variáveis de entrada
        scaler = StandardScaler()
        dados_normalizados = scaler.fit_transform(data)

        # Divide os dados em treino e teste
        X_train, X_test, y_train, y_test = train_test_split(
            dados_normalizados,
            target_encoded,
            test_size=0.2,
            random_state=0,
            stratify=target_encoded  # Mantém a proporção das classes
        )
        return X_train, X_test, y_train, y_test, None, None  # Retorna os dados e as classes codificadas



---

### Método train_and_evaluate

A função train_and_evaluate é responsável por treinar e avaliar um modelo de rede neural, passando por várias etapas, como carregar dados, configurar o modelo, treinar e avaliar sua performance.

1. **Exibição do tipo de tarefa**: Exibe uma mensagem indicando qual tipo de tarefa (classificação binária, regressão ou classificação multiclasse) o modelo vai realizar.

2. **Carregar os dados**

- Chama a função load_data que carrega os dados de um arquivo CSV especificado pelo dataset_path. Ela retorna:
- X_train e X_test: dados de entrada (variáveis independentes) para treino e teste.
- y_train e y_test: os rótulos (variáveis dependentes) para treino e teste.
- y_mean e y_std: valores de média e desvio padrão para normalização, caso a tarefa seja de regressão.

3. **Configuração do modelo**: O modelo é configurado com os parâmetros.


In [ ]:
def train_and_evaluate(dataset_path, task_type, activation, loss, output_neurons, hidden_neurons, learning_rate, epochs=1500):
    print(f"\n=== Treinando modelo para tarefa: {task_type} ===")
    
    # Carregar os dados
    X_train, X_test, y_train, y_test, y_mean, y_std = load_data(dataset_path, task_type)
    
    # Configurar o modelo
    model = NeuralNetwork(
        input_size=X_train.shape[1],
        hidden_neurons=hidden_neurons,
        output_neurons=output_neurons,
        activation=activation,
        loss=loss,
        learning_rate=learning_rate,
        task_type=task_type
    )

A parte seguinte do código é responsável por treinar e avaliar o modelo de rede neural.

1. **Treinamento do modelo**

- `model.train(...)`: Chama o método train da classe NeuralNetwork, passando os dados de treino (X_train e y_train) e os dados de teste (X_test e y_test).
- `epochs=epochs`: Define o número de épocas (iterações) para o treinamento do modelo. O valor de epochs é passado como argumento na chamada da função train_and_evaluate, e determina quantas vezes o modelo será treinado com os dados de entrada.

2. **Avaliação do modelo**

- `model.evaluate(...)`: Chama o método evaluate da classe NeuralNetwork para avaliar o desempenho do modelo usando os dados de teste (X_test e y_test).
- y_mean e y_std: São passados para o método evaluate caso a tarefa seja de regressão, para permitir que o modelo denormalize os valores preditos.
- A função evaluate retorna o desempenho do modelo, como precisão (para tarefas de classificação) ou erro (para regressão), e esses resultados são armazenados na variável results.

3. **Retorna os dados**

In [ ]:
    # Treinar o modelo
    model.train(X_train, y_train, X_test, y_test, epochs=epochs)
    
    # Avaliar o modelo
    results = model.evaluate(X_test, y_test, y_mean=y_mean, y_std=y_std)
    return results

O próximo e último bloco de código está dentro de uma estrutura condicional `if __name__ == "__main__"`:, o que significa que ele será executado apenas quando o script for executado diretamente (não importado como módulo).

A função executa o treinamento e avaliação de três modelos diferentes, cada um com um tipo de tarefa distinto.

In [ ]:
if __name__ == "__main__":  # Verifica se o script está sendo executado diretamente
    # Treinar e avaliar os 3 modelos
    
    # 1. Classificação Binária
    train_and_evaluate(
        dataset_path="heart binary.csv",  # Caminho para o arquivo de dados de classificação binária
        task_type='binary',  # Define que a tarefa é de classificação binária
        activation='sigmoid',  # Função de ativação sigmoid, apropriada para tarefas binárias
        loss='binary',  # Função de perda para classificação binária
        output_neurons=1,  # Número de neurônios na camada de saída (apenas 1, pois é binário)
        hidden_neurons=200,  # Número de neurônios na camada oculta
        learning_rate=0.005  # Taxa de aprendizado
    )

    # 2. Regressão
    train_and_evaluate(
        dataset_path="advertisement.csv",  # Caminho para o arquivo de dados de regressão
        task_type='regression',  # Define que a tarefa é de regressão (valor contínuo)
        activation='linear',  # Função de ativação linear, comum para regressão
        loss='regression',  # Função de perda para regressão
        output_neurons=1,  # Número de neurônios na camada de saída (1 valor contínuo)
        hidden_neurons=7,  # Número de neurônios na camada oculta
        learning_rate=0.005  # Taxa de aprendizado
    )

    # 3. Classificação Multiclasse
    train_and_evaluate(
        dataset_path="iris.csv",  # Caminho para o arquivo de dados de classificação multiclasse
        task_type='multiclass',  # Define que a tarefa é de classificação multiclasse
        activation='relu',  # Função de ativação ReLU, comum em redes neurais profundas
        loss='multiclass',  # Função de perda para classificação multiclasse
        output_neurons=3,  # Número de neurônios na camada de saída (3 classes possíveis)
        hidden_neurons=5,  # Número de neurônios na camada oculta
        learning_rate=0.015  # Taxa de aprendizado
    )
